In [2]:
# Импорт библиотек
from pathlib import Path
from warnings import filterwarnings
import os
import sys

from IPython.display import display, Markdown
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


#from arch import arch_model
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, accuracy
from surprise import BaselineOnly

from scipy.sparse import csr_matrix
from scipy.optimize import minimize, least_squares



from lightfm import LightFM
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k    

import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
#import pmdarima as pm
import statsmodels.graphics.tsaplots as sgt
import scipy
import sklearn

import time

# # Попытка применить ускорение Intel Extension для scikit-learn
# try:
#     from sklearnex import patch_sklearn, config_context
#     patch_sklearn()
#     print('Intel Extension для scikit-learn успешно активирован')
# except (ImportError, ModuleNotFoundError, Exception) as e:
#     print(f'Не удалось активировать Intel Extension для scikit-learn: {e}')
#     print('   Продолжаем работу без ускорения (это нормально)')

In [3]:
filterwarnings("ignore")
# warnings.filterwarnings('ignore', category=UserWarning)
# warnings.filterwarnings('ignore', message='.*ConvergenceWarning.*')

plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")
%matplotlib inline


SEED = 42

# Проверка текущей рабочей директории
print(f"Текущая рабочая директория: {os.getcwd()}")

# Текущая рабочая директория (скорее всего .../IDE)
current_dir = os.getcwd()

# Относительный путь от корня проекта до папки с ноутбуком
# Обратите внимание: используем r'' для корректной обработки слешей и пробелов
relative_path_to_notebook = r'skillfactory/MATH_ML_15'

# Проверяем, не перешли ли мы уже в нужную папку (чтобы не было ошибок при перезапуске ячейки)
if not current_dir.endswith("MATH_ML_15"):
    # Собираем полный путь
    new_dir = os.path.join(current_dir, relative_path_to_notebook)
    
    # Меняем рабочую директорию
    try:
        os.chdir(new_dir)
        print(f"Рабочая директория изменена на: {os.getcwd()}")
    except FileNotFoundError:
        print("Ошибка: путь не найден. Проверьте правильность названий папок.")
else:
    print("Рабочая директория уже верная.")

Текущая рабочая директория: /home/pavel/IDE
Рабочая директория изменена на: /home/pavel/IDE/skillfactory/MATH_ML_15


In [4]:
ratings = pd.read_csv('./data/Gooddreadbooks/ratings.csv') # Поставленные оценки
books = pd.read_csv('./data/Gooddreadbooks/books.csv') # Информация о книгах
tags = pd.read_csv('./data/Gooddreadbooks/tags.csv') # Информация о тегах
book_tags = pd.read_csv('./data/Gooddreadbooks/book_tags.csv') # Книги с тегами 

display(ratings.head())
display(books.head())
display(tags.head())
display(book_tags.head())

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


,tag_id,tag_name
0,509,19th-century
1,923,20th-century
2,941,21st-century
3,1499,abuse
4,1540,action


,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [5]:
#4.1
dict_map = dict(zip(books.goodreads_book_id,books.book_id))
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: dict_map[x])
 
book_tags[book_tags['goodreads_book_id']==5]

,goodreads_book_id,tag_id,count,id
300,5,11557,40087,18
301,5,11305,39330,18
302,5,8717,17944,18
303,5,33114,12856,18
304,5,30574,11909,18
...,...,...,...,...
395,5,20781,299,18
396,5,32345,298,18
397,5,12600,282,18
398,5,3379,277,18


In [6]:
# 4.2
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags.shape

(300738, 4)

In [7]:
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы),
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))

In [8]:
ratings_matrix.shape

(53425, 10001)

In [9]:
meta_matrix = csr_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id))) 

In [10]:
meta_matrix.shape

(10001, 33269)

In [11]:
# 4.4 
ratings_matrix.mean()

0.007086188900997592

In [12]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

In [13]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

print(f"Разбиение завершено успешно!\nTrain shape: {train.shape}\nTest shape: {test.shape}")

print(f"Всего оценок: {ratings_matrix.nnz}")
print(f"Оценок в Train: {train.nnz} ({train.nnz / ratings_matrix.nnz:.1%})")
print(f"Оценок в Test:  {test.nnz} ({test.nnz / ratings_matrix.nnz:.1%})")

Разбиение завершено успешно!
Train shape: (53425, 10001)
Test shape: (53425, 10001)
Всего оценок: 979478
Оценок в Train: 783582 (80.0%)
Оценок в Test:  195896 (20.0%)


In [14]:
model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    num_threads = 6, # Количество потоков
    verbose=True # Отображение обучения
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 100%|██████████| 10/10 [01:17<00:00,  7.76s/it]


In [15]:
# 4.5
prec_score = precision_at_k(
                     model,
                     test,
                     item_features = meta_matrix).mean() 
print(prec_score)

# Для предсказания рейтинга для нового пользователя можно воспользоваться методом predict():
#scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items),user_features=new_user_feature)


0.023747211


0.023387857